In [3]:
import math as m

def getWDL(score):
    return (score[0] > score[1]) - (score[0] < score[1])

def difference_factor(correct, voorspeld):
    doelsaldo_delta = abs(abs(correct[0]-correct[1]) - abs(voorspeld[0]-voorspeld[1]))
    doelpunten_delta = abs((correct[0]+correct[1]) - (voorspeld[0]+voorspeld[1]))
    return m.pi**(-2*(
        doelsaldo_delta**2/max(abs(voorspeld[0]-voorspeld[1]),2)
        +doelpunten_delta**2/max(abs(voorspeld[0]+voorspeld[1]),2)
      ))


def calc_bonuspoints(correct, voorspeld):
    if (correct[0] - correct[1]) * (voorspeld[0] - voorspeld[1]) < 0:
        return 0
    doelpunten = [2, 1, 3, 7, 10, 17, 27, 44, 71, 115, 186]
    totaal_doelpunten = [0,0,0,1,3,6,10,15,21,28,36,45,55,66,78,91,105,120,136,153,171]
    thuis = doelpunten[correct[0]]
    uit = doelpunten[correct[1]]
    totaal = totaal_doelpunten[correct[0]+correct[1]]
    return m.floor(difference_factor(correct, voorspeld) * (thuis + uit + totaal))

def calc_basepoints(correct, voorspeld):
    if getWDL(correct) != getWDL(voorspeld):
        return 0
    return 7 if getWDL(correct) == 0 else 5

def calc_points(correct, voorspeld):
    return calc_basepoints(correct, voorspeld) + calc_bonuspoints(correct, voorspeld)  


In [4]:
teams = ['ARG', 'AUS', 'BEL', 'BRA', 'CAN', 'CMR', 'CRC', 'CRO', 'DEN', 'ECU', 'ENG', 'ESP', 'FRA', 'GER', 'GHA', 'IRN', 'JPN', 'KOR', 'KSA', 'MAR', 'MEX', 'NED', 'POL', 'POR', 'QAT', 'SEN', 'SRB', 'SUI', 'TUN', 'URU', 'USA', 'WAL']

In [5]:
import json
with open("data_kuleuven.json", "r") as f:
    k = f.read()
data = json.loads(k)
print(data["NED-QAT"])


[[5.523708680762211, 1.7720753024918359, 0.9288295976376811, 0.3245627392299642, 0.08505944359792242, 0.02148491100307942], [4.871115905383919, 9.022767226707465, 2.7470095108221346, 0.9722131553864098, 0.2580205748204244, 0.06615008974422064], [7.018246653625766, 7.551034485718895, 5.993321004855852, 1.455408983955826, 0.3910965856123104, 0.10175806287551252], [6.741204965196053, 7.346053293042494, 4.0006572184697164, 2.098660434959765, 0.3949632076426702, 0.10427852733891402], [4.856324516534342, 5.359116651002198, 2.955130101580551, 1.0856827359900099, 0.4613509172810382, 0.08008816010186649], [4.977750488944864, 5.608749406635435, 3.157601677398483, 1.1842765142272276, 0.3328993740262128, 0.13026673983908918]]


In [6]:
def result_prob(prob, ngoals_home, ngoals_away, nmax=9):
        factor_home = 1
        factor_away = 1
        if ngoals_home >= 5:
            factor_home = [0.5**(i) for i in range(1,nmax-5+2)][ngoals_home-5]
        if ngoals_away >= 5:
            factor_away = [0.5**(i) for i in range(1,nmax-5+2)][ngoals_away-5]
        return prob[min(ngoals_home,5)][min(ngoals_away,5)]*factor_home*factor_away

def getOptimalPronostiek(home: str, away: str):            
    prob = data.get(f"{home}-{away}", None)
    if prob is None:
        return [((k[1],k[0]), v) for k,v in getOptimalPronostiek(away, home)]
    scores = {}
    best_prono = (0, 0)
    for i in range(10):
        for j in range(10):
            voorspeld = (i, j)
            score = 0
            for k in range(10):
                for l in range(10):
                    correct = (k, l)
                    score += calc_points(correct, voorspeld)*result_prob(prob, k, l)
            scores[voorspeld] = score
    return sorted(list(scores.items()), reverse=True, key=lambda n:n[1])
            
    

In [41]:
groups = [
    ["QAT", "ECU", "SEN", "NED"],
    ["ENG", "IRN", "USA", "WAL"],
    ["ARG", "KSA", "MEX", "POL"],
    ["FRA", "AUS", "DEN", "TUN"],
    ["ESP", "CRC", "GER", "JPN"],
    ["BEL", "CAN", "MAR", "CRO"],
    ["BRA", "SRB", "SUI", "CMR"],
    ["POR", "GHA", "URU", "KOR"]
]
def getMatch(i):
    return [(0,1), (2,3), (0,2), (3,1), (3,0), (1,2)][i]
for id,group in zip("ABCDEFGH",groups):
    for i in range(6):
        home, away = getMatch(i)
        print(f"{id}{i+1}: {group[home]}-{group[away]}")
        print(getOptimalPronostiek(group[home], group[away])[0:5])

A1: QAT-ECU
[((2, 3), 357.01516742861673), ((1, 4), 352.53443864686403), ((2, 4), 346.7991777607126), ((3, 4), 334.4778866367956), ((1, 3), 332.4205018986046)]
A2: SEN-NED
[((1, 4), 365.71379948969457), ((2, 3), 359.69584671219815), ((2, 4), 349.9486828660174), ((1, 3), 346.6016306393311), ((1, 5), 342.1207884097571)]
A3: QAT-SEN
[((1, 4), 345.79783388010713), ((2, 3), 340.78397154824444), ((1, 3), 333.5029667079287), ((2, 4), 327.64663047014415), ((0, 4), 320.4561594500471)]
A4: NED-ECU
[((4, 1), 428.81604726404174), ((5, 1), 424.6982689446094), ((4, 2), 415.25201839016836), ((5, 2), 412.51326463942144), ((3, 2), 407.27442783701923)]
A5: NED-QAT
[((5, 1), 543.3661180110784), ((6, 1), 526.8476221062795), ((4, 1), 522.9001300873975), ((7, 1), 509.04094599645214), ((5, 0), 493.66991694003633)]
A6: ECU-SEN
[((3, 3), 319.9601402621164), ((2, 2), 305.96475888524435), ((2, 3), 285.48332620996393), ((4, 4), 268.7392591684176), ((2, 4), 261.8100694758735)]
B1: ENG-IRN
[((4, 1), 365.08119177086

In [9]:
print(f"NED-ARG")
print(getOptimalPronostiek("NED", "ARG")[0:5])
print(f"CRO-BRA")
print(getOptimalPronostiek("CRO", "BRA")[0:5])
print(f"ENG-FRA")
print(getOptimalPronostiek("ENG", "FRA")[0:5])
print(f"MAR-POR")
print(getOptimalPronostiek("MAR", "POR")[0:5])

NED-ARG
[((2, 3), 321.967224280462), ((3, 3), 320.5095447182678), ((1, 4), 304.8042758172233), ((2, 4), 303.8189313392198), ((2, 2), 302.53251557086605)]
CRO-BRA
[((1, 4), 449.98461200179935), ((1, 5), 438.263733622547), ((1, 6), 418.12567623906773), ((0, 4), 416.55645094370357), ((2, 4), 411.52323007027115)]
ENG-FRA
[((3, 3), 322.6363007429179), ((2, 2), 310.72546151505117), ((4, 4), 271.9030850768033), ((2, 3), 271.9025572536715), ((3, 2), 250.70708843378378)]
MAR-POR
[((2, 3), 340.7382843938989), ((1, 4), 335.52581898107496), ((2, 4), 326.7087878070405), ((1, 3), 321.7001869624763), ((3, 4), 313.936912111227)]


In [13]:
print(f"NED-USA")
print(getOptimalPronostiek("NED", "USA")[0:5])
print(f"ENG-SEN")
print(getOptimalPronostiek("ENG", "SEN")[0:5])
print(f"ARG-AUS")
print(getOptimalPronostiek("ARG", "AUS")[0:5])
print(f"FRA-POL")
print(getOptimalPronostiek("FRA", "POL")[0:5])
print(f"JPN-CRO")
print(getOptimalPronostiek("JPN", "CRO")[0:5])
print(f"MAR-ESP")
print(getOptimalPronostiek("MAR", "ESP")[0:5])
print(f"BRA-KOR")
print(getOptimalPronostiek("BRA", "KOR")[0:5])
print(f"POR-SUI")
print(getOptimalPronostiek("POR", "SUI")[0:5])

NED-USA
[((4, 1), 372.18255212882184), ((3, 2), 366.7955347699435), ((4, 2), 359.04870359347), ((5, 1), 351.0853349603993), ((3, 1), 349.3480902242775)]
ENG-SEN
[((4, 1), 356.0624079737404), ((3, 2), 345.1724108257728), ((3, 1), 342.63855075865166), ((4, 2), 333.44074116131435), ((4, 0), 333.26727824589796)]
ARG-AUS
[((5, 1), 637.9710669491204), ((6, 1), 629.022724237266), ((5, 0), 615.380047799088), ((6, 0), 607.1948472757815), ((7, 1), 607.0229372490281)]
FRA-POL
[((4, 1), 446.0465147972685), ((5, 1), 443.1992762454005), ((6, 1), 424.50329972563446), ((4, 2), 423.7102064079425), ((5, 2), 423.08786650286277)]
JPN-CRO
[((2, 3), 335.2595264635806), ((1, 4), 322.33167202578886), ((3, 3), 320.3777062218989), ((2, 4), 319.8167855429297), ((3, 4), 311.4917375051733)]
MAR-ESP
[((1, 4), 423.3367091795609), ((1, 5), 409.213928644722), ((2, 4), 396.86494281767534), ((2, 3), 395.3768528626807), ((1, 6), 389.6268064245465)]
BRA-KOR
[((4, 1), 464.36121093719686), ((4, 0), 454.7562180907237), ((5, 